<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/azure_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q loguru textract tiktoken openai azure-ai-ml mlflow azureml-sdk azureml-mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.0/814.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.6 MB/s eta 0:00

# before running this notebook, UPLOAD these files
- openai.env
- azure.env
- samples Resumes stored in dir of same name
 

In [2]:
import os,argparse,loguru, json, time, datetime
from pathlib import Path
from loguru import logger

In [3]:
@logger.catch
def maybe_load_aml_env_vars(json_env_path=None):
  import os, json
  if not json_env_path: return False

  try:
    with open(json_env_path, "r") as f:
      env_vars = json.load(f)
    os.environ["resource_group"] = env_vars["resource_group"]
    os.environ["workspace_name"] = env_vars["workspace_name"]
    os.environ["subscription_id"] = env_vars["subscription_id"]

  except Exception as e:
    logger.error(f"{e}")

  if (
      os.getenv("RESOURCE_NAME") 
      and os.getenv("WORKSPACE_NAME")
      and os.getenv("SUBSCRIPTION_ID")
      ):
    return True
  else: 
    return False

In [ ]:
# @logger.catch
# def aml_env_connected():
#   "use environ vars to connect to Azure ML"
#   pass

# @logger.catch
# def maybe_start_experiment(AZURE_AVAILABLE):
#   if AZURE_AVAILABLE:
#     pass

# @logger.catch
# def maybe_stop_experiment(AZURE_AVAILABLE):
#   if AZURE_AVAILABLE:
#     pass


In [10]:
def set_open_ai_key(key_path=None):
  import json, os
  from pathlib import Path
  if not key_path: return False

  try:
    with open(key_path, "r") as f:
        env_vars = json.load(f)
    os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    openai.Model.list() #test a random command on the openai API
    return True
  except Exception as e:
    logger.error(e)
  finally:
    return False

def test_set_open_ai_key(key_path=None):
  openai.api_key = None #disconnect from api key if already registered
  try:
    set_open_ai_key(key_path)
    openai.Model.list()
    return True
  except Exception as e:
    logger.error(e)
    return False



In [5]:
# sdk v1 needed here? 
# but see https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow-configure-tracking?view=azureml-api-2&tabs=python%2Cmlflow
from azureml.core import Workspace 
import mlflow

ws = Workspace.from_config("azure.env")
tracking_uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(tracking_uri)

# sdk v2 needed here
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()


maybe_load_aml_env_vars("azure.env")

# get a handle to the workspace
ml_client = MLClient(
    subscription_id=os.getenv("subscription_id"),
    resource_group_name=os.getenv("resource_group"),
    workspace_name=os.getenv("workspace_name"),
    credential=credential,
)


# Check azure ml is working correctly
experiment_name = 'mlflow-2'
mlflow.set_experiment(experiment_name)
from random import random

with mlflow.start_run() as mlflow_run:
    mlflow.log_param("hello_param", "world")
    mlflow.log_metric("hello_metric2", random())
    os.system(f"echo 'hello world2' > helloworld2.txt")
    mlflow.log_artifact("helloworld2.txt")


Performing interactive authentication. Please follow the instructions on the terminal.


Interactive authentication successfully completed.


Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [6]:
cwd = Path.cwd()
resume_path = cwd / "Resumes"
resume_path.mkdir(exist_ok=True)



## 1 OBTAIN *DATASET*




### extract text from pdf

In [7]:
import textract
import os
import openai
import tiktoken

In [8]:
set_open_ai_key(cwd / "openai.env")

True

In [11]:
#TODO walk the directory to get all the filenames, use as names of the people in tagging and document retrieval
file_names = ['SeanReed2023.pdf'] #["Jesse_Jayant.pdf", "Nadia_Smythe.pdf", "jin_wang.pdf" ,"sarah_jones.pdf"]
file_paths = [(resume_path / file) for file in file_names];print(file_paths)
names = [path.stem.lower() for path in file_paths];names

# Extract the raw text from each PDF using textract
# TODO Extract the file name and save it in a separate name field, or save these using name as dictkey, or save in dataframe row
texts =[textract.process((file_path), method='pdfminer').decode('utf-8') for file_path in file_paths]

texts = [text.strip().replace("  ", " ") for text in texts]
clean_texts = [item1+" NameOnResume: "+ item2 for item1,item2 in zip(texts, names)]

len(clean_texts[0]),clean_texts[0][-50:]

[PosixPath('/content/Resumes/SeanReed2023.pdf')]


['seanreed2023']

# SPLIT SECTIONS VERSION #2
source: Embedding_Wikipedia_articles_for_search.ipynb 
https://colab.research.google.com/drive/1EJMtCmF8jZc2Y-c1RaBxFSCTPcjzjJf4#scrollTo=TOVSYkDur9zA

Next, we'll recursively split long sections into smaller sections.

There's no perfect recipe for splitting text into sections.

Some tradeoffs include:
- Longer sections may be better for questions that require more context
- Longer sections may be worse for retrieval, as they may have more topics muddled together
- Shorter sections are better for reducing costs (which are proportional to the number of tokens)
- Shorter sections allow more sections to be retrieved, which may help with recall
- Overlapping sections may help prevent answers from being cut by section boundaries

Here, we'll use a simple approach and limit sections to 1,600 tokens each, recursively halving any sections that are too long. To avoid cutting in the middle of useful sentences, we'll split along paragraph boundaries when possible.

In [32]:
#TODO walk the directory to get all the filenames, use as names of the people in tagging and document retrieval
file_names = ["SeanReed2023.pdf","Jesse_Jayant.pdf", "Nadia_Smythe.pdf", "jin_wang.pdf" ,"sarah_jones.pdf"]
file_paths = [(resume_path / file) for file in file_names];print(file_paths)
names = [path.stem.lower() for path in file_paths];names

# Extract the raw text from each PDF using textract
# TODO Extract the file name and save it in a separate name field, or save these using name as dictkey, or save in dataframe row
texts =[textract.process((file_path), method='pdfminer').decode('utf-8') for file_path in file_paths]

texts = [text.strip().replace("  ", " ") for text in texts]
clean_texts = [(['NameOnResume: '+ item1], item2) for item1,item2 in zip(names,texts)]

len(clean_texts)

[PosixPath('/content/Resumes/SeanReed2023.pdf'), PosixPath('/content/Resumes/Jesse_Jayant.pdf'), PosixPath('/content/Resumes/Nadia_Smythe.pdf'), PosixPath('/content/Resumes/jin_wang.pdf'), PosixPath('/content/Resumes/sarah_jones.pdf')]


5

In [33]:
GPT_MODEL = 'text-ada-001'  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Split a string in two, on a delimiter, trying to balance tokens on each side."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # no delimiter found
    elif len(chunks) == 2:
        return chunks  # no need to search for halfway point
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]


def truncated_string(
    string: str,
    model: str,
    max_tokens: int,
    print_warning: bool = True,
) -> str:
    """Truncate a string to a maximum number of tokens."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        logger.warning(f"Warning: Truncated string from {len(encoded_string)} tokens to {max_tokens} tokens.")
    return truncated_string

In [37]:
def split_strings_from_subsection(
    subsection: tuple[list[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> list[str]:
    """
    Split a subsection into a list of subsections, each with no more than max_tokens.
    Each subsection is a tuple of parent titles [H1, H2, ...] and text (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # if length is fine, return string
    if num_tokens_in_string <= max_tokens:
        return [string]
    # if recursion hasn't found a split after X iterations, just truncate
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # otherwise, split in half and recurse
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # if either half is empty, retry with a more fine-grained delimiter
                continue
            else:
                # recurse on each half
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,
                    )
                    results.extend(half_strings)
                return results
    # otherwise no split was found, so just truncate (should be very rare)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]
 

In [40]:
# split resumes into chunks
MAX_TOKENS = 900
resume_strings = []
for section in clean_texts:
    resume_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(clean_texts)} resumes split into {len(resume_strings)} strings.")


5 resumes split into 8 strings.


In [41]:
for item in resume_strings:
  print(len(item), num_tokens(item))

2842 648
2117 528
2540 635
2682 654
2840 753
2957 779
2618 618
2380 637


In [42]:
# Example prompt - 
document = '<document>'
template_prompt=f'''Extract the value of the NameOnResume fields from this document.
  \n\nDocument: \"\"\"{document}\"\"\"\n\n'''
print(template_prompt)

Extract the value of the NameOnResume fields from this document.
  

Document: """<document>"""


